In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from flask import Flask, request, jsonify
from model import load_data, encode_data, calculate_optimal_plan, calculate_water_intake

app = Flask(__name__)

# Load and prepare the data
data = load_data()
data, label_encoders = encode_data(data)

# Load the pre-trained models
clf = joblib.load('fitness_goal_classifier.pkl')
regressor = joblib.load('calories_burned_regressor.pkl')
scaler = joblib.load('scaler.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()

    # Step 1: Prepare inputs for Level 1 Prediction (Fitness Goal)
    gender_encoded = label_encoders['Gender'].transform([data['gender']])[0]
    workout_type_encoded = label_encoders['Workout_Type'].transform([data['workout_type']])[0]
    level1_input = [[data['age'], gender_encoded, data['bmi'], data['workout_frequency'],
                     data['experience_level'], workout_type_encoded, data['fat_percentage']]]

    # Level 1 Prediction (Fitness Goal)
    fitness_goal_pred = clf.predict(level1_input)[0]
    fitness_goal_label = label_encoders['Fitness_Goal'].inverse_transform([fitness_goal_pred])[0]

    # Step 2: Add the predicted fitness goal to features for level 2 (Calories Burned)
    level2_input = scaler.transform([[data['age'], gender_encoded, data['weight'], data['height'], data['max_bpm'],
                                       data['avg_bpm'], data['resting_bpm'], data['session_duration'],
                                       workout_type_encoded, data['fat_percentage'], data['bmi'], fitness_goal_pred]])

    # Level 2 Prediction (Calories Burned)
    calories_burned_pred = regressor.predict(level2_input)[0]

    # Step 3: Use the results from Level 1 and Level 2 for Level 3 Optimization (Workout Plan)
    optimal_plan = calculate_optimal_plan(user_experience=data['experience_level'], max_bpm=data['max_bpm'],
                                          predicted_calories=calories_burned_pred, age=data['age'], fitness_goal=fitness_goal_label)
    optimal_bpm = optimal_plan[1]

    if optimal_bpm > 160:
        optimal_workout_type = 'HIIT'
    elif optimal_bpm > 140:
        optimal_workout_type = 'Cardio'
    elif optimal_bpm > 120:
        optimal_workout_type = 'Strength'
    else:
        optimal_workout_type = 'Endurance'

    # Calculate recommended water intake
    recommended_water = calculate_water_intake(data['weight'], optimal_plan[0])

    # Return the results
    return jsonify({
        "fitness_goal": fitness_goal_label,
        "calories_burned": calories_burned_pred,
        "optimal_plan": optimal_plan.tolist(),
        "optimal_workout_type": optimal_workout_type,
        "recommended_water_intake": recommended_water
    })

if __name__ == '__main__':
    app.run(debug=True)

Level 1 Training Accuracy: 0.9370179948586118
Level 1 Testing Accuracy: 0.9179487179487179
Level 1 Cross-Validation Accuracy (mean): 0.9105617504733854
Level 2 Training R^2: 0.9964858218046424
Level 2 Testing R^2: 0.9728080212914191
Level 2 Training MSE: 252.84283800610535
Level 2 Testing MSE: 2268.4915209935903
Level 2 Training MAE: 11.901571336760924
Level 2 Testing MAE: 35.73239743589743
Optimal Workout Plan:
 Session Duration: 2.00 hours,  Target BPM: 100.0,  Fatigue Risk: 0.00
Recommended Workout Type: Endurance
